In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tensorflow_probability as tfp

_NUMERICAL_PRECISION_CONCENTRATION = 3  # Keep concentration values up to 3 decimals

MEAN = 0.0
N_POINTS = int(1e5)

concentrations = np.logspace(-3, 6, N_POINTS)
# concentrations = np.linspace(0.0, MAX_CONCENTRATION, N_POINTS)
concentrations[0] = 0.0

## Compute standard deviations 

In [ ]:
%autoreload 2

# Compute variance from samples

N_SAMPLES = 100
means = np.repeat(MEAN, N_POINTS)
vm_dist = tfp.distributions.VonMises(loc=means, concentration=concentrations)
phases_samples = vm_dist.sample(N_SAMPLES)
phases_samples = np.mod(phases_samples + np.pi, 2 * np.pi) - np.pi
phases_samples_var = np.mean(np.power(phases_samples, 2), axis=0)
phases_samples_std = np.sqrt(phases_samples_var)
phases_samples_std

In [ ]:
%autoreload 2

# Compute phase variance from integral

import scipy as scp

def get_exp_scaled_function_to_integrate(concentration: float):
    """
    Get the exponentially scaled function to integrate for computing the variance.
    Original function: x^2 * exp(concentration * cos(x))
    Exponentially scaled: exp(-concentration) x^2 * exp(concentration * cos(x))
    i.e. x^2 * exp(concentration * (cos(x) - 1))
    """
    def _aux(x):
        return np.power(x, 2) * np.exp(concentration * (np.cos(x) - 1))
    return _aux

def log_bessel(x):
    """Numerically stable computation of log(I_0(x))"""
    return np.log(scp.special.i0e(x)) + x

phases_std = []
for con in concentrations:
    # Compute exponentially scaled integral for numerical stability
    exp_scaled_integ = scp.integrate.quad(get_exp_scaled_function_to_integrate(con), 0, np.pi)
    # Get non-exp scaled log-variance
    log_variance = con + np.log(exp_scaled_integ[0] / np.pi) - log_bessel(con)
    variance = np.exp(log_variance)
    phases_std.append(np.sqrt(variance))

phases_std = np.array(phases_std, dtype=np.float64)

In [ ]:
%autoreload 2

import scipy as scp
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
# ax.plot(concentrations, phases_samples_std)
ax.set_xscale("log")
ax.plot(concentrations, phases_std)

## Store (phase std, von Mises concentration) array

In [ ]:
%autoreload 2

import numpy as np

from src.utils.save_utils import SafeOpen
from study.calibration.utils import VON_MISES_STD_CONCENTRATION_MAPPING_FOLDER, VON_MISES_STD_CONCENTRATION_ARRAY_FILENAME

std_concentration_array = np.array([concentrations, phases_std])
std_concentration_array = np.transpose(std_concentration_array)

with SafeOpen(VON_MISES_STD_CONCENTRATION_MAPPING_FOLDER, VON_MISES_STD_CONCENTRATION_ARRAY_FILENAME, "wb") as file:
    np.save(file, std_concentration_array)

## Select (std, concentration) mapping for experiments

In [ ]:
%autoreload 2

N_SELECTED_STD = 10

min_std = min(phases_std)
max_std = max(phases_std)


selected_std = np.linspace(min_std, max_std, N_SELECTED_STD, endpoint=True)

print(f"Selected std angles (deg): {selected_std * (180 / np.pi)}")
selected_std

In [ ]:
%autoreload 2

indexes_concentration = []
errors = []
for std in selected_std:
    std_error = np.abs(phases_std - std)
    idx = np.argmin(std_error)
    indexes_concentration.append(idx)
    errors.append(std_error[idx])

indexes_concentration = np.array(indexes_concentration, dtype=np.int64)
errors = np.array(errors, dtype=np.float32)

selected_concentrations = concentrations[indexes_concentration]

print(f"Selected concentrations: {selected_concentrations}")
print(f"With std errors: {errors}")

fig, ax = plt.subplots()
ax.plot(selected_std, selected_concentrations, marker="o")
ax.set_yscale("log")

In [ ]:
np.array([1,2,5, np.nan])


## Final mapping and save

In [ ]:
%autoreload 2

import os
import json
from src.utils.save_utils import SafeOpen
from study.calibration.utils import VON_MISES_STD_CONCENTRATION_MAPPING_FOLDER, get_von_mises_std_concentration_mapping_filename


perfect_phase_mapping = {"std": 0.0, "concentration": "_infinity"}
von_mises_std_concentration_mapping = [
    {
        "std": std,
        "concentration": round(concentration, _NUMERICAL_PRECISION_CONCENTRATION)
    }
    for std, concentration in zip(selected_std, selected_concentrations)
]
# Replace min std (should be very close to 0.0) with perfect phase scenario
print(f"Replacing mapping {von_mises_std_concentration_mapping[0]} with {perfect_phase_mapping}")
von_mises_std_concentration_mapping[0] = perfect_phase_mapping

with SafeOpen(VON_MISES_STD_CONCENTRATION_MAPPING_FOLDER, get_von_mises_std_concentration_mapping_filename(N_SELECTED_STD), "w") as file:
    json.dump(von_mises_std_concentration_mapping, file)